In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/accre-gpu-jobs-2022-v2.csv', error_bad_lines=False)


b'Skipping line 124084: expected 12 fields, saw 13\nSkipping line 124809: expected 12 fields, saw 13\n'


In [3]:
df.head()

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED


In [4]:
df.shape

(336950, 12)

In [5]:
df.PARTITION.value_counts()

maxwell    128228
pascal     125751
turing      82971
Name: PARTITION, dtype: int64

In [6]:
df.STATE.unique()

array(['COMPLETED', 'CANCELLED by 9201', 'CANCELLED by 859023',
       'CANCELLED by 153500', 'CANCELLED by 168618',
       'CANCELLED by 888717', 'FAILED', 'CANCELLED by 859269',
       'CANCELLED by 756088', 'CANCELLED by 887337',
       'CANCELLED by 858683', 'CANCELLED', 'CANCELLED by 885613',
       'CANCELLED by 342288', 'CANCELLED by 855431',
       'CANCELLED by 491383', 'CANCELLED by 776393',
       'CANCELLED by 844796', 'CANCELLED by 780767',
       'CANCELLED by 505355', 'CANCELLED by 649321',
       'CANCELLED by 890189', 'CANCELLED by 59229', 'CANCELLED by 649319',
       'CANCELLED by 590520', 'CANCELLED by 478095',
       'CANCELLED by 196703', 'CANCELLED by 308082',
       'CANCELLED by 757496', 'CANCELLED by 454080',
       'CANCELLED by 512947', 'CANCELLED by 762866',
       'CANCELLED by 859465', 'CANCELLED by 161909',
       'CANCELLED by 785491', 'CANCELLED by 90423', 'CANCELLED by 159541',
       'CANCELLED by 859439', 'CANCELLED by 349259',
       'CANCELLED by 

ACCRE has three scheduler partitions that groups can get access to which consist of machines each with 4 GPU cards. Each partition is named after the nvidia processor generation that it contains, from oldest to newest: maxwell, pascal, and turing. GPU nodes are requested per-GPU, and users can request up to 4 GPUs per node. For each GPU the user is allocated up to one quarter of the machine's RAM and CPU cores. All of the current GPU-accelerated hardware is also connected to a special high-speed RoCE (RDMA over converged ethernet) network allowing for memory sharing on large-scale multi-GPU jobs, so users can also request multiple servers for analysis that requires more than four GPU cards.

The main objectives of this project are to examine the following questions:
1. What is the distribution of per-GPU main memory usage over all runtime-weighed jobs in each partition? Knowing this will help ACCRE to understand our users memory needs for future hardware purchases.
2. What is the distribution of the number of GPUs in each job (runtime-weighted) for each partition? What fraction of runtime-weighted and GPU-weighted jobs are using more than 4 GPUs and thus probably using the RoCE networking? Is this fraction different for each partition?
3. What is the total runtime usage per-gpu (i.e. multiply runtime by the number of gpus) in each of the 3 partitions over the last year?
4. What is the distribution of different groups and users accessing each partition? In each partition, who are the top users, and do they represent a majority of the runtime-weighted jobs on the partition?
5. Currently there is a 5 day limit on runtime for GPU jobs, although some users have been asking for extensions. What is the distribution of requested runtime and actual runtime on jobs on each partition? Do users really need more time, or are they simply always requesting the maximum?

In [7]:
#weigh jobs by runtime(usedtime), and show distribution of memory usage per GPU

In [8]:
#GPU
df.GPUS.unique()

array([ 1,  2,  4,  3,  8, 32, 24, 16, 18, 12,  6, 40, 60, 30, 20],
      dtype=int64)

In [9]:
df.GPUS.describe()

count    336950.000000
mean          1.088734
std           0.868473
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          60.000000
Name: GPUS, dtype: float64

In [10]:
df.USEDMEM.describe()

count     336950
unique    114062
top            0
freq       58747
Name: USEDMEM, dtype: object

In [11]:
df.USEDMEM

0          18.66M
1               0
2         402.32M
3         417.62M
4         450.09M
           ...   
336945          0
336946          0
336947          0
336948    145.24M
336949          0
Name: USEDMEM, Length: 336950, dtype: object

In [12]:
df['USEDMEM']= df["USEDMEM"].str.replace("M","")

In [13]:
df.head()

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED


In [14]:
df.dtypes

JOBID        object
ACCOUNT      object
USER         object
USEDMEM      object
REQTIME      object
USEDTIME     object
NODES         int64
CPUS          int64
GPUS          int64
PARTITION    object
EXITCODE     object
STATE        object
dtype: object

In [15]:
df.USEDTIME.describe()

count       336950
unique       20818
top       00:00:00
freq         51083
Name: USEDTIME, dtype: object

In [16]:
df[df['USEDTIME'].str.contains('-')]

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
6,32880722,glasshouse,brady,0,3-12:00:00,3-12:00:07,1,1,1,pascal,0:0,COMPLETED
327,32884086,glasshouse,miya,374.89,5-00:00:00,1-08:38:34,1,1,1,pascal,0:0,COMPLETED
540,32885557,malanga,alwilda,7572.38,2-00:00:00,1-05:49:07,1,2,1,pascal,0:0,COMPLETED
784,32888937,virginia,almond,0,2-12:00:00,2-12:00:10,1,18,3,turing,0:0,COMPLETED
1004,32891492,glasshouse,erick,241.82,4-04:00:00,4-04:00:26,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336412,46636477,glasshouse,erick,0,4-04:00:00,1-22:51:47,1,1,1,pascal,0:0,RUNNING
336413,46636478,glasshouse,erick,0,4-04:00:00,1-22:51:47,1,1,1,pascal,0:0,RUNNING
336414,46636479,glasshouse,erick,0,4-04:00:00,1-22:51:47,1,1,1,pascal,0:0,RUNNING
336456,46647371,glasshouse,mel,0,3-08:00:00,1-09:55:00,1,20,8,turing,0:0,RUNNING


In [17]:
#replace - in usedtime with days

df.USEDTIME= df.USEDTIME.str.replace('-','days ',regex=True)

In [18]:
df['USEDTIMEtd'] = pd.to_timedelta(df['USEDTIME'])

In [24]:
df.head(5)

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE,USEDTIMEtd
0,32880657,malanga,arline,18.66,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED,0 days 00:13:13
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED,0 days 00:00:22
2,32880718,glasshouse,brady,402.32,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED,0 days 00:08:57
3,32880719,glasshouse,brady,417.62,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED,0 days 05:09:46
4,32880720,glasshouse,brady,450.09,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED,0 days 04:11:10


In [25]:
df['USEDTIME_SEC'] = df['USEDTIMEtd'].dt.total_seconds()

In [26]:
df.head()

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE,USEDTIMEtd,USEDTIME_SEC
0,32880657,malanga,arline,18.66,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED,0 days 00:13:13,793.0
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED,0 days 00:00:22,22.0
2,32880718,glasshouse,brady,402.32,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED,0 days 00:08:57,537.0
3,32880719,glasshouse,brady,417.62,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED,0 days 05:09:46,18586.0
4,32880720,glasshouse,brady,450.09,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED,0 days 04:11:10,15070.0


In [27]:
df.USEDMEM= df.USEDMEM.str.replace('M','',regex=True)

In [28]:
df.USEDMEM = df['USEDMEM'].str.strip()

In [29]:
df.STATE.unique()

array(['COMPLETED', 'CANCELLED by 9201', 'CANCELLED by 859023',
       'CANCELLED by 153500', 'CANCELLED by 168618',
       'CANCELLED by 888717', 'FAILED', 'CANCELLED by 859269',
       'CANCELLED by 756088', 'CANCELLED by 887337',
       'CANCELLED by 858683', 'CANCELLED', 'CANCELLED by 885613',
       'CANCELLED by 342288', 'CANCELLED by 855431',
       'CANCELLED by 491383', 'CANCELLED by 776393',
       'CANCELLED by 844796', 'CANCELLED by 780767',
       'CANCELLED by 505355', 'CANCELLED by 649321',
       'CANCELLED by 890189', 'CANCELLED by 59229', 'CANCELLED by 649319',
       'CANCELLED by 590520', 'CANCELLED by 478095',
       'CANCELLED by 196703', 'CANCELLED by 308082',
       'CANCELLED by 757496', 'CANCELLED by 454080',
       'CANCELLED by 512947', 'CANCELLED by 762866',
       'CANCELLED by 859465', 'CANCELLED by 161909',
       'CANCELLED by 785491', 'CANCELLED by 90423', 'CANCELLED by 159541',
       'CANCELLED by 859439', 'CANCELLED by 349259',
       'CANCELLED by 

In [30]:
df.USEDMEM= df.USEDMEM.str.replace('M','',regex=True)
df.USEDMEM = df['USEDMEM'].str.strip()

In [31]:
df.dtypes

JOBID                    object
ACCOUNT                  object
USER                     object
USEDMEM                  object
REQTIME                  object
USEDTIME                 object
NODES                     int64
CPUS                      int64
GPUS                      int64
PARTITION                object
EXITCODE                 object
STATE                    object
USEDTIMEtd      timedelta64[ns]
USEDTIME_SEC            float64
dtype: object

In [32]:
#df_test['USEDMEM'] = df_test['USEDMEM'].astype('float')
#df_test['USEDMEM_n'] = pd.to_numeric(df['USEDMEM'])
df = df.astype({"USEDMEM":'float'})

In [33]:
df.dtypes

JOBID                    object
ACCOUNT                  object
USER                     object
USEDMEM                 float64
REQTIME                  object
USEDTIME                 object
NODES                     int64
CPUS                      int64
GPUS                      int64
PARTITION                object
EXITCODE                 object
STATE                    object
USEDTIMEtd      timedelta64[ns]
USEDTIME_SEC            float64
dtype: object

In [34]:
df_test = df[(df['STATE'] == 'COMPLETED') & (df['USEDTIME_SEC'] != 0)]

In [35]:
df_test

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE,USEDTIMEtd,USEDTIME_SEC
0,32880657,malanga,arline,18.66,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED,0 days 00:13:13,793.0
1,32880701,glasshouse,brady,0.00,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED,0 days 00:00:22,22.0
2,32880718,glasshouse,brady,402.32,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED,0 days 00:08:57,537.0
3,32880719,glasshouse,brady,417.62,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED,0 days 05:09:46,18586.0
4,32880720,glasshouse,brady,450.09,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED,0 days 04:11:10,15070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336933,46753813,harovin,shannon,137.96,5-00:00:00,00:00:52,1,1,1,turing,0:0,COMPLETED,0 days 00:00:52,52.0
336934,46753820_26,harovin,porsha,5093.70,5-00:00:00,00:34:51,1,6,1,turing,0:0,COMPLETED,0 days 00:34:51,2091.0
336936,46753823,harovin,shannon,156.21,5-00:00:00,00:04:03,1,1,1,turing,0:0,COMPLETED,0 days 00:04:03,243.0
336942,46754095,harovin,shannon,2820.52,5-00:00:00,00:11:13,1,1,1,turing,0:0,COMPLETED,0 days 00:11:13,673.0


In [36]:
df_test[df_test['GPUS'] == 60]

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE,USEDTIMEtd,USEDTIME_SEC
62122,37020263,pasilla,dola,2.53,00:20:00,00:00:03,15,120,60,pascal,0:0,COMPLETED,0 days 00:00:03,3.0
270301,44976228,pasilla,dola,7.41,00:20:00,00:00:02,15,120,60,pascal,0:0,COMPLETED,0 days 00:00:02,2.0


In [37]:
df_test.USEDTIME_SEC

0           793.0
1            22.0
2           537.0
3         18586.0
4         15070.0
           ...   
336933       52.0
336934     2091.0
336936      243.0
336942      673.0
336948       86.0
Name: USEDTIME_SEC, Length: 283288, dtype: float64

1. What is the distribution of per-GPU main memory usage over all runtime-weighed jobs in each partition?

In [38]:
#Create new column 'RT Weighted Memory', weight of each memory by runtime
df['RT_WEIGHTED_MEM'] = df['USEDMEM']*df['USEDTIME_SEC']/(df['USEDTIME_SEC'].sum())

In [39]:
df['USEDTIME_SEC'].sum()

6749129955.0

In [40]:
df.RT_WEIGHTED_MEM.describe()

count    3.369500e+05
mean     4.892904e-03
std      9.302397e-02
min      0.000000e+00
25%      6.554919e-09
50%      2.833740e-05
75%      8.736953e-05
max      1.413326e+01
Name: RT_WEIGHTED_MEM, dtype: float64

In [41]:
#Try new column without dividing out by time
df['RT_USEDMEM'] = df['USEDMEM']*df['USEDTIME_SEC']

In [42]:
df.RT_USEDMEM.describe()

count    3.369500e+05
mean     3.302285e+07
std      6.278309e+08
min      0.000000e+00
25%      4.424000e+01
50%      1.912528e+05
75%      5.896683e+05
max      9.538720e+10
Name: RT_USEDMEM, dtype: float64

In [43]:
x = df.groupby(['PARTITION'])['RT_WEIGHTED_MEM'].agg(['sum'])
#sum of rt_used mem, then divide that by the sum of usedtime

In [44]:
x

,sum
PARTITION,
maxwell,188.532022
pascal,549.513699
turing,910.618391


In [45]:
df['MEM_PER_GPU'] = df['RT_USEDMEM']/df['GPUS']


In [46]:
df.head()

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE,USEDTIMEtd,USEDTIME_SEC,RT_WEIGHTED_MEM,RT_USEDMEM,MEM_PER_GPU
0,32880657,malanga,arline,18.66,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED,0 days 00:13:13,793.0,0.000002,14797.38,14797.38
1,32880701,glasshouse,brady,0.00,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED,0 days 00:00:22,22.0,0.000000,0.00,0.00
2,32880718,glasshouse,brady,402.32,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED,0 days 00:08:57,537.0,0.000032,216045.84,216045.84
3,32880719,glasshouse,brady,417.62,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED,0 days 05:09:46,18586.0,0.001150,7761885.32,7761885.32
4,32880720,glasshouse,brady,450.09,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED,0 days 04:11:10,15070.0,0.001005,6782856.30,6782856.30


In [ ]:
#make new column 'weighted_memory'
#df['WEIGHTED_MEMORY'] = df['USEDMEM']/df['GPUS']

In [ ]:
#DF['WEIGHTED_MEMORY'] = d['MEM']/df['GPUS']*df['USEDTIME']

In [ ]:
#w = df.groupby('PARTITION')['weighted_memory'].agg('sum')
#turing uses more memory and for longer--the average went up when accounting for how long-the memory usage is much higher
#job using more memory and time is more influential, 
#weighted and unweighted - when not accounting for length, pascal and maxwell much shorter times
##calculate duration/job
#maxwell more popular but jobs run much shorter
#distribution of different states


